In [3]:
from __future__ import unicode_literals
import numpy as np
from scipy.signal import hilbert
from scipy import integrate
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d


'''
En este script vamos a ocupar un método pseudoespectral para resolver la ecuacion de GL:

dA/dt = A - (b3 - i)|A|^2*A + (1 + i *b1)*d^2 A/dx^2

'''

#Parametros del problema
dt = 0.1
dx = 1.0
N = 500 #Grid points

L = N*dx
T = N*dt


x = np.arange(N)*dx 
xx = np.arange(2*N)*dx
xxx = np.arange(-N,N)
t = np.arange(N)*dt

### Canvas, aqui dibujaremos la simulacion
Canvas = np.zeros((N,N), dtype=complex)
Canvas_fourier = np.zeros((N,N), dtype=complex)


### Calcula el termino cubico de la ecuacion de GL en Fourier
def cubic_fourier(c_actual):
    c_actual[N/3:2*N/3] = 0
    A_actual = np.fft.ifft(c_actual)
    cubic_term = A_actual*np.abs(A_actual)**2
    return np.fft.fft(cubic_term)
    

### Pasamos la ecuacion al espacio de Fourier:
##  c_n(t) = f(c_n(t))
##  Con c_n(t), la transformada de Fourier de A

def GL_fourier(c_actual,b1,b3):
    cubic_term = cubic_fourier(c_actual)
    return c_actual - (b3-1j)*cubic_term - (1+b1*1j)*k**2*c_actual

## Reolvemos el problema con RK4
def time_step(c_actual,b1,b3):
    k1 = GL_fourier(c_actual,b1,b3)
    k2 = GL_fourier(.5*k1*dt,b1,b3)
    k3 = GL_fourier(.5*k2*dt,b1,b3)
    k4 = GL_fourier(k3*dt,b1,b3)
    return c_actual + dt/6.0*(k1+ 2*k2 + 2*k3 + k4)

def winding_number(phase):
    integrand = np.gradient(phase,dx)
    return 1/(2*np.pi) * integrate.simps(y = phase, dx = dx)

b3 = .3
#B1 = np.arange(-1.2,-.6,0.01)
B1 = np.array([-1.2])

for b1 in B1:
    ### Condiciones iniciales
    A_inicial = np.random.rand(N)*np.exp(1j*np.random.uniform(low = 0, high= 2*np.pi,size=N))
    c_inicial = np.fft.fft(A_inicial)
    c_new = c_inicial
    k = 2*np.pi*np.fft.fftfreq(c_inicial.size, dx)
    for _ in range(100000):
        c_new = time_step(c_new,b1,b3)

    for i  in range(N):
        for _ in range(50):
            c_new = time_step(c_new,b1,b3)
        Canvas_fourier[i,:] = c_new
    
    Canvas = np.fft.ifft(Canvas_fourier)
    phase = np.angle(Canvas)
    np.save('simulacionesGL/phase/phase_b1='+str(b1)+'_b3='+str(b3), phase)
    np.save('simulacionesGL/Canvas/Canvas_b1='+str(b1)+'_b3='+str(b3), Canvas)





ImportError: cannot import name sigtools

In [2]:
b1 = -1.1
b3 = .3

phase = np.load('simulacionesGL/phase/phase_b1='+str(b1)+'_b3='+str(b3)+'.npy')
Canvas = np.load('simulacionesGL/Canvas/Canvas_b1='+str(b1)+'_b3='+str(b3)+'.npy')
envelope = np.abs(Canvas)

fig = plt.figure(figsize = (110,47))
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(221)

im1 = ax1.imshow(envelope, aspect='equal', cmap='summer', origin='lower')
ax1.set_xlabel(r'$x$', fontsize=45, x = .95)
ax1.set_ylabel(r'$t$', fontsize=45, y = .9, rotation=0)

cb1=plt.colorbar(im1)


im2 = ax2.imshow(phase, aspect='equal', cmap='Spectral', origin='lower')
ax2.set_xlabel(r'$x$', fontsize=45, x = .95)
ax2.set_ylabel(r'$t$', fontsize=45, y = .9, rotation=0)
cb2 = plt.colorbar(im2)
plt.show()

NameError: name 'plt' is not defined

array([[-0.16835128, -0.15543869, -0.14251041, ..., -0.20699472,
        -0.19412932, -0.18124816],
       [ 1.73633517,  1.74924686,  1.76217424, ...,  1.69769443,
         1.71055893,  1.72343919],
       [-2.64216364, -2.62925285, -2.61632638, ..., -2.68080167,
        -2.66793807, -2.65505872],
       ..., 
       [-2.29407306, -2.28159396, -2.26909964, ..., -2.3314191 ,
        -2.31898563, -2.30653695],
       [-0.38936253, -0.37688426, -0.36439078, ..., -0.42670606,
        -0.41427342, -0.40182558],
       [ 1.51534805,  1.52782548,  1.54031813, ...,  1.47800702,
         1.49043882,  1.50288583]])

In [26]:
from scipy.optimize import curve_fit
### Desplaza la funcion u en l
def shift(u_actual, l):
    uu = np.concatenate([u_actual, u_actual])
    if l >=0:
        u = interp1d(xx, uu , kind='cubic')
    else:
        u = interp1d(xxx, uu , kind='cubic')
    x_new = x + l
    return u(x_new)

def corr(phase, l):
    phase_new = np.apply_along_axis(shift, 1, phase, l)
    integrand = np.zeros(N)
    for i in range(len(phase)):
        integrand[i] = integrate.simps(phase[i]*phase_new[i], dx = dx) - L*np.mean(phase[i])**2
    return 1/(L*T) * integrate.simps(integrand, dx=dt)




In [109]:
b3 = 0.5
#B1 = np.arange(-1.2,-.4,0.01)
B1 = np.arange(0.4,0.7,.01) #para b3 = .5
X = np.arange(1,15, 0.2)
m = len(X)
Corr_matrix = []
a_parameter = []

def fit(x, a, b):
    return -a*x + b

for b1 in B1:
    phase = np.load('simulacionesGL/phase/phase_b1='+str(b1)+'_b3='+str(b3)+'.npy')
    #Canvas = np.load('simulacionesGL/Canvas/Canvas_b1='+str(b1)+'_b3='+str(b3)+'.npy')
    correlation = np.zeros(m)
    for i in range(m):
        correlation[i] = corr(phase, X[i])
    Corr_matrix.append(correlation)
    index = np.where(correlation>1e-1)
    a_parameter.append(curve_fit(fit, X[index], np.log(correlation[index]))[0][0])
    print b1



0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69


In [120]:
a_parameter = np.array(a_parameter)
Corr_matrix = np.array(Corr_matrix)
np.save('simulacionesGl/a_parameter_b3='+str(b3), a_parameter)
np.save('simulacionesGl/Corr_matrix_b3='+str(b3), Corr_matrix)






In [2]:
b3 = .3
B1 = np.arange(-1.2,-.6,0.01)
a_parameter = np.load('simulacionesGl/a_parameter_b3='+str(b3)+'.npy')
Corr_matrix = np.load('simulacionesGl/Corr_matrix_b3='+str(b3)+'.npy')

In [3]:
len(a_parameter)

30

ValueError: x and y must have same first dimension, but have shapes (60L,) and (30L,)

array([], dtype=int32)